# Variational Inference
Sources
* [David Blei talk on Variational Inference](https://www.youtube.com/watch?v=Dv86zdWjJKQ)
* Blei, et al. (2017) Variational Inference: A Review for Statisticians
* [Python code for variational inference](https://zhiyzuo.github.io/VI/#python-implementation)

Variational inference is a general method used to approximate a posterior: that's it. The posterior inference problem is: given a probability model (including observed and hidden variables), we square the model up with observed data. After discovering patterns, we can predict and explore. This allows customized data analysis. It is particularly useful because we can draw up a graphical model and implement it. It separates our assumptions from computation and application.

A probabilistic model is a joint distribution $$ p\left(z,x\right) $$ where $x$ are observed variables and $z$ are latent (hidden) variables. We can make inference about the hidden variables through the posterior, which is the conditional distibution $$ p\left(z\middle| x\right) = p\left(z,x\right) / p\left(x\right). $$

Main issue: the evidence $p\left(x\right)$ is generally untractable, as an integral over the latent space.

**Variational inference** (VI) is one approach to approximating the posterior, and it does so via optimization. We posit a family $$ \mathcal{Q} = \left\{ q\left(z; \theta \right) \right\} $$ of distributions $q$ over the latent space. Each distribution in the family is parametrized by some variational parameters $\theta$. In VI we optimize over the variational parameters $\theta$ to find the "best" approximation to the posterior.

This seems difficult because we only have observed data. How can we possibly make this work?

**Stochastic optimization** techniques are particularly useful because they scale VI up to big data and generalize to a large class of models.

See: graphical models. Node: random variable; arrow: dependence between random variables; shaded node: observed random variable; blank node: latent variable; plate (rectangle): repetition.


### Conditionally conjugate models

Observations $x_i$, local variables $z=z_{1:n}$, global variables $\beta$. The $i$th data point $x_i$ depends only on $z_i$ and $\beta$. We have the joint distribution $$ p\left(\beta,z,x\right) = p\left(\beta\right) \prod_{i=1}^n p\left(z_i,x_i \, \middle| \,\beta \right).$$ Note that $\beta$ is conditionally independent from $x_i$ and $z_i$. The goal is to calculate the posterior $$ p\left(\beta,z \, \middle|\, x\right)$$ A **complete conditional** is the conditional of a latent variable given the observations and other latent variables. Assume that each complete conditional is in the exponential family, i.e., $ p\left(z_i\, \middle| \, \beta, x_i\right) $ and $p\left(\beta \, \middle| \, z,x\right)$ are in the exponential family. Given these assumptions, we can make claims about the parameters of these complete conditional distributions. 

These are important because many common models fall into this category:
* Bayesian mixture models
* time-series models
* matrix factorization
* Dirichlet process mixtures
* multi-level regression
* stochastic block models


### Variational inference

We want to minimize the KL divergence between our variational family and the true posterior. Then we will have our approximation. Unfortunately, the KL divergence contains the evidence term $\mathbb{E}\left[ \log p\left(x\right)\right]$ and so is intractable (can't compute the integral in the evidence). Instead, **we optimize using the evidence lower bound (ELBO)**. By expanding ELBO, we see that it balances two terms: $$ \mathcal{L}\left(\theta\right) = \mathbb{E}_q\left[\log\, p\left(\beta,z,x\right)\right]-\mathbb{E}_q\left[\log\, q\left(\beta,z\, \middle| \,\theta\right)\right]$$ We maximize the ELBO. The first term encourages $q$ to place its mass on the MAP estimate (i.e., seek parameters which give high likelihood to the data), and the second encourages $q$ to be diffuse (spread around). ELBO is non-convex!


### One form for $q\left(\beta,z\right)$: the mean-field family

The mean-field family is a fully-factorized distribution: $$ q\left(\beta, z ; \lambda, \phi\right) = q\left(\beta; \lambda\right) \prod_{i=1}^n q\left(z_i;\phi_i\right) $$ (Note here that the quantities after the semicolons are the parameters of the distributions). Each factor is the same family as the model's complete conditional: $$ p\left(\beta\, \middle| \, z,x\right) = h\left(\beta\right) \exp \left\{ \eta_g\left(z,x\right)^T \beta - a\left( \eta_g\left(z,x\right)\right)\right\}$$ $$q\left(\beta;\lambda\right) = h\left(\beta\right)\exp\left\{\lambda^T\beta - a\left(\lambda\right)\right\}$$ This is a bunch of disconnected variables. Every variable is disconnected from every other. Through the ELBO, we are connecting this distribution to the posterior we care about. We will never capture posterior correlations, since these don't show up in the $q$.

We can now optimize the ELBO using coordinate ascent.

How are the expectation values computed?

### Stochastic optimization

Main idea: replace expensive gradient computation with a noisy, cheap (local) version.

### Black-box variational inference 

This is the ultimate goal, whereby we can take any data and any model and simply throw them into a variational inference black box.

# Follow notes at [Stanford CS228 page](https://ermongroup.github.io/cs228-notes/inference/variational/)

Inference in probabilistic models is usually difficult because the integrals involved are intractible. Instead, we often resort to sampling methods such as MCMC; Gibbs sampling and Metropolis-Hastings are two particular MCMC methods. These are nice because they are guaranteed to be globally optimal, but they scale poorly and can take a lot of time.

A different approach is to perform variational inference (VI), which poses inference as an optimization problem.

VI methods will only find locally optimal solutions, but we can check for convergence. They also scale better and allow for stochastic gradient optimization, parallelization, and GPU acceleration.

### KL divergence
The KL divergence between two distributions $q$ and $p$ is $$ \text{KL}\left(q\middle\| p\right) = \sum_{x} q\left(x\right) \log \frac{q\left(x\right)}{p\left(x\right)} $$ Important properties are that $\text{KL}\left(q\middle\|p\right)\geq 0$ and equality is obtained iff $q=p$. Note: it is *asymmetric*, and thus not an actual metric.

How can this be used for inference? 

Assume that our distribution $p\left(x_1,x_2,\ldots,x_n;\theta\right)$ is of the form $$ p\left(x_1,x_2,\ldots,x_n;\theta\right) = \frac{\prod_{k=1}^n \phi_k\left(x_k;\theta\right)}{Z\left(\theta\right)} = \frac{\tilde{p}\left(x_1,\ldots,x_n;\theta\right)}{Z\left(\theta\right)}$$ where $\phi_k$ are factors and $Z$ is a normalization constant. This constant is the difficult part, since it is generally intractable. What if we ignore this constant and try to work with the un-normalized distribution $ \tilde{p}\left(x\right) = \prod_k \phi_k\left(x_k;\theta\right)$? Look at $$ \begin{align} J\left(q\right) & = \sum_x q\left(x\right) \log \frac{q\left(x\right)}{\tilde{p}\left(x\right)} \\ & = \sum_x q\left(x\right) \log \frac{q\left(x\right)}{p\left(x\right)} - \log Z\left(\theta\right)  \\ & = \text{KL}\left(q \middle\| p\right) - \log Z\left(\theta\right) \end{align} $$ so that $J$ is literally just the $\text{KL}$ divergence with the troublesome constant subtracted out.

Do some simple rearranging: $$ \log Z\left(\theta\right) = \text{KL}\left(q \middle\| p\right) - J\left(q\right) \geq -J\left(q\right) $$ since $\text{KL} \geq 0$. Thus $-J$ is a lower bound on the log partition function $\log Z\left(\theta\right)$. In the special case where we are seeking the marginal probability of vars $x$ given observations $D$, $$ p\left(x\,\middle| \, D\right) = \frac{p\left(x,D\right)}{p\left(D\right)}, $$ we have observed data $D$ acting as the evidence. Then minimizing $J\left(q\right)$ is equivalent to maximizing this lower bound on the log-likelihood (?) $\log p\left(D\right)$ of the observed data. 

Thus $-J\left(q\right)$ is the **evidence lower bound (ELBO)**. Wer can write it as $$ \begin{align} -J\left(q\right) & = \sum_x q\left(x\right) \log \frac{\tilde{p}\left(x\right)}{q\left(x\right)} \\ & = \sum_x q\left(x\right) \log \tilde{p}\left(x\right) - \sum_x q\left(x\right) \log q\left(x\right) \\ & = \mathbb{E}_q \left[ \log \tilde{p}\left(x\right)\right]-\mathbb{E}_q \left[ \log q\left(x\right)\right] \\ & = \mathbb{E}_q \left[ \log \tilde{p}\left(x\right) - \log q\left(x\right)\right] \\ & \leq \log Z\left(\theta\right) \end{align} $$ Recall that $\theta$ are the distribution parameters of our model which have yet to be determined. From the above relation, we have $$ \log Z\left(\theta\right) = \text{KL}\left(q\middle\|p\right) + \text{ELBO}\left(q\right) \geq \text{ELBO}\left(q\right) $$ Thus, by maximizing the ELBO we are pinching the $\text{KL}$ divergence and forcing it to shrink, since $\log Z\left(\theta\right)$ is a constant.

### Asymmetry of KLD
We chose $q$ as the first "input" to the KL divergence because the expectation values all become with respect to $q$. We would not be able to take the expectations with respect to $p$, in general. This has some interesting implications with regard to the different types of distributions which can be fit in the possible cases.

### Mean-field inference

What distributions $q$ will we use? Common choices include the exponential family, neural networks, Gaussian processes, latent variable models, etc. Think about how these could be used here?

A common family of distributions $q\left(\cdot\right)$ is the mean-field family, which are fully-factored distributions: $$ q\left(x\right) = q_1\left(x_1\right)q_2\left(x_2\right)\cdots q_n\left(x_n\right) = \prod_{i=1}^n q_i\left(x_i\right) $$ In this case, inference becomes the optimization problem $$ \min_{q_1,q_2,\ldots, q_n} J\left(q\right) $$ A straightforward way to perform this is via coordinate descent over the $q_i$. That is, iterate through the $q_i$ and optimize $J$ over each in turn, keeping the remaining coordinates $q_j, j\neq i,$ constant (Note: this is the CAVI algorithm).

Where does the update formula come from? Write the ELBO as a function of only the $j$-th variational factor $q_j$, and absorb the rest into a constant. $$ \begin{align} ELBO\left(q_j\right) & = \mathbb{E}_q \left[ \log \tilde{p}\right] - \mathbb{E}_q \left[ \log q \right] \\ & = \sum_x q \log \tilde{p} - \sum_x q \log q  \\ & = \sum_x \left(q_j \log \tilde{p}\right) \left(\prod_{i\neq j} q_i\right) - \sum_x q_j \left(\prod_{i\neq j} q_i\right) \log \left[ q_j \left(\prod_{i\neq j} q_i\right) \right]  \\ & = \sum_x \left(q_j \log \tilde{p}\right) \left(q_{-j}\right) - \sum_x q_j q_{-j} \left[ \log q_j +  \log q_{-j} \right]   \end{align} $$